In [1]:
#Computes the accuracies for the outputs from the EACL 2017 experiments on
#joint incremental utterance segmentation and disfluency detection
#this assumes the experiments are in simple_rnn_disf/rnn_disf_detection/experiments/
from __future__ import division
%matplotlib inline
import pandas as pd
import numpy as np
import sys
from collections import defaultdict
import matplotlib.pyplot as plt
from copy import deepcopy
# from mumodo.mumodoIO import open_intervalframe_from_textgrid

In [2]:
#add the evaluation module functions
sys.path.append("../../../../")
from deep_disfluency.evaluation.disf_evaluation import incremental_output_disfluency_eval_from_file
from deep_disfluency.evaluation.disf_evaluation import final_output_disfluency_eval_from_file, ACCURACY_HEADER
from deep_disfluency.evaluation.eval_utils import get_tag_data_from_corpus_file
from deep_disfluency.evaluation.results_utils import convert_to_latex

In [3]:
top_dir = "../../../../deep_disfluency"
experiment_dir = top_dir + "/experiments"
#experiment_dir = "/home/dsg-labuser/Desktop/rnn_experiments/"
all_results = {}
all_error_dicts = {}

In [4]:
#the evaluation files (as text files)
disf_dir = top_dir + "/data/disfluency_detection/switchboard"
disfluency_files = [
                    disf_dir+"/swbd_heldout_partial_timings_data.csv",
                    disf_dir+"/swbd_heldout_partial_timings_data.csv",
                    disf_dir+"/swbd_test_partial_timings_data.csv",
                    disf_dir+"/swbd_test_partial_timings_data.csv"
                    ]
    
dialogue_speakers = []
good_asr_test = [line.strip("\n") for line in open(top_dir + \
            "/data/disfluency_detection/swda_divisions_disfluency_detection/SWDisfTestASRgood_ranges.text")]
good_asr_heldout = [line.strip("\n") for line in open(top_dir + \
        "/data/disfluency_detection/swda_divisions_disfluency_detection/SWDisfHeldoutASRgood_ranges.text")]
#Need comparable stopping criterion
#1. Weighted F-score all tags  or   loss all tags - have this for all
#2. Unweighted F-score all tags would need to recompute for all / per-class loss - would need to recompute for 33
#3. Complex rps + tto combined metric as explored - would need to recompute for all, though is closer
#allsystemsfinal = [(33,45,'RNN'),(35,6,'LSTM'),\
#                   (36,15,'LSTM (complex tags) (timing)'),\
#                   (34,37,'RNN (complex tags) (timing)')]
allsystemsfinal = [
    (33,45,'RNN (timing)'),(35,6,'LSTM (timing)'),
    (37,6,'LSTM (disf only) (timing)'),\
     (38,8,'LSTM (TTO only) (timing)'),\
              #    (39,2,'LSTM (complex tags)'),\
        ]

for exp,e,system in allsystemsfinal:
    print exp,e,system
    #if 'complex' in system: break
    hyp_dir = experiment_dir + "/0{0}/".format(exp)
    for division, disf_file in zip(["heldout", "heldout_asr", "test", "test_asr"],disfluency_files):
        #if not division == "heldout": continue
        print division
        IDs, timings, words, pos_tags, labels = get_tag_data_from_corpus_file(disf_file)
        #dialogue_speakers.extend(sort_into_dialogue_speakers(IDs,mappings,utts, pos_tags, labels))
        if "heldout" in division:
            good_asr = good_asr_heldout
        else:
            good_asr = good_asr_test
        gold_data = {} #map from the file name to the data
        for dialogue,a,b,c,d in zip(IDs, timings, words, pos_tags, labels):
            if "asr" in division and not dialogue[:4] in good_asr: continue
            gold_data[dialogue] = (a,b,c,d)
        if "asr" in division:
            error = False
            word = False
        else:
            error = True
            word = True
        #the below does just the final output evaluation, assuming a final output file, faster
        #results,speaker_rate_dict,error_analysis = final_output_disfluency_eval_from_file(
                                    #hyp_dir+"epoch_{0}/predictions_inc_{1}.final".format(e,division),\
                                    #gold_data, utt_eval=True, error_analysis=error, word=word, interval=True,\
                                    #outputfilename=None)
        #the below does incremental and final output in one, also outputting the final outputs
        #derivable from the incremental output, takes quite a while
        results,speaker_rate_dict,error_analysis = incremental_output_disfluency_eval_from_file(
                                                        hyp_dir+"epoch_{0}/predictions_inc_{1}.increco"
                                                            .format(e,division),\
                                                        gold_data, 
                                                        utt_eval=True, 
                                                        error_analysis=error, 
                                                        word=word, 
                                                        interval=True,\
                                                        outputfilename=hyp_dir+\
                                                            "epoch_{0}/predictions_inc_{1}.final"
                                                            .format(e,division))
        for k,v in results.items():
            print k,v
        all_results[division + "_" + system] = deepcopy(results)
        all_error_dicts[division + "_" + system] = deepcopy(error_analysis)
    #break #just one system for now

33 45 RNN
heldout
loading data ../../../../rnn_disf_detection//data/disfluency_detection/switchboard/swbd_heldout_partial_timings_data.csv
loaded 104 sequences
104 speakers
incremental output disfluency evaluation
word= True interval= True utt_eval= True


/home/julian/.local/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


final output disfluency evaluation
word= True interval= True utt_eval= True
word
interval
r_<rm.<i.<rp_interval 0.244318848636
pearson_r_p_value_rps_number 2.47371064803e-66
p_<rms_interval 0
f1_<rm_word 0
t_t_detection_<rps_interval 0.238689839572
f1_<rpnsub_word 0
r_<rpnsub_word 0
r_t>_interval 0
p_<i_interval 0.409424379233
pearson_r_p_value_rps_rate_per_word 6.80692621974e-55
f1_<rpnrep_interval 0
r_<e_interval 0.914806512382
NIST_SU_interval 53.0006978367
f1_<rps_relaxed_interval 0.773198482933
    p_<rps_relaxed_word None
    delayed_acc_<rm_mean_interval None
f1_<rpndel_word 0
f1_<rpndel_interval 0
r_<rpnrep_word 0
p_<rm.<i.<rp_word 0.79677877783
    delayed_acc_<rm_mean_word None
f1_<rpn_interval 0
    p_<e_interval None
t_t_detection_t>_word 0.335757575758
f1_<rp_interval 0.46382101234
delayed_acc_<rm_3_word None
r_<rm.<i.<rp_word 0.235014670952
f1_<rm_interval 0
r_<i_word 0.362745098039
t_t_detection_<rms_interval nan
    f1_<rm_word None
p_<rps_interval 0.480109081441
delaye

In [8]:
#print out the results
for div,results in all_results.items():
    print div, "************************************"
    #all_results[div+ "(complex tags)"] = results
    for r,v in results.items():
        #if v == 0 or v == None: continue
        if not "word" in r: continue
        #if not r[:2] in ["p_","r_"]: continue
        print r,v

test_asr_RNN ************************************
pearson_r_p_value_rps_rate_per_word 0.000623282689123
pearson_r_correl_rps_rate_per_word 0.55696149754
test_RNN ************************************
r_<rpnsub_word 0
pearson_r_p_value_rps_rate_per_word 2.02881566551e-36
    p_<rps_relaxed_word None
p_t>_relaxed_word 0
r_<rm.<i.<rp_word 0.204464738711
    f1_<rm_word None
f1_<rp_word 0.505660377358
DSER_word 58.0947511929
r_<e_word 0.892080536913
p_<rps_word 0.787564766839
    delayed_acc_<rm_2_word None
    p_<e_relaxed_word None
edit_overhead_rel_word 11.0275421465
r_<rps_word 0.60283286119
    p_<e_word None
f1_<rpn_word 0
p_t>_word 0.80844012373
f1_<e_relaxed_word 0
p_<rp_word 0.79348630644
NIST_SU_word 52.4199045671
p_<rps_relaxed_word 0.854182087343
r_<rm.<rp.<i_word None
r_<rp_word 0.371062651436
    delayed_acc_<rm_4_word None
p_<i_word 0.685
f1_<rm.<i.<rp_word 0.323954983923
p_<rm_word 0
f1_t>_word 0.704060034635
    delayed_acc_<rm_5_word None
p_<rpnrep_word 0
f1_<rms_word 0
f1

In [9]:
#filter out the heldout files
test_only = [x for x in all_results.keys() if not "heldout" in x and not "complex" in x]
print test_only
test_results = {k : all_results[k] for k in test_only}
#have a look at the test results
final = convert_to_latex(test_results)
final = final.drop(final.columns[[-2]], axis=1)
final

['test_asr_RNN', 'test_RNN']


,System (eval. method),$F_{rps}$ (per word),$F_{rps}$ (per 10s window),$F_{e}$ (per word),$F_{e}$ (per 10s window),$F_{TTO}$ (per word),$F_{TTO}$ (per 10s window),DSER (word)
0,RNN (transcript),0.683,0.730,0.909,0.886,0.704,0.710,58.09
1,RNN (ASR results),-,0.542,-,0.718,-,0.681,-


In [10]:
inc = convert_to_latex(test_results, inc=True)
inc

,System (eval. method),TTD$_{rps}$ (word),TTD$_{rps}$ (time in s),TTD$_{tto}$ (word),TTD$_{tto}$ (time in s),EO
0,RNN (transcript),1.007,0.253,0.350,1.814,11.03
1,RNN (ASR results),-,0.198,-,3.649,20.53


In [ ]:
#Write the final and incremental output to a file
latex_file = open("latex_output_single_tasks_simple.txt","w")
latex_file.write(inc.to_latex(index=False))
latex_file.write("\n\n" + final.to_latex(index=False))
latex_file.close()

In [ ]:
print error

In [11]:
#Error analyses
error = {"TP" : {}, "FP" : {}, "FN": {} }
 
for div,all_error in all_error_dicts.items():
    print div, type(all_error)
    if type(all_error) == bool: continue
    if "test" in div: continue
    #if not 'TTO only' in div or "asr" in div: continue
    for tag,errors in all_error.items():
        print div, tag
        #if not 'TTO only' in div or "asr" in div: continue
        if not tag == "t>": continue
        for k,v in errors.items():
    
            print ""
            #if k == "FP": continue
            print k, len(v)
            typedict = defaultdict(int)
            lendict = defaultdict(int)
            print v[0]
            for repair in v:
                #if len(repair)==0: continue
                #print "*"
                #print repair
                #print repair.gold_context
                onset = ""
                if tag == "<rps":
                    
                    for i in rcange(0,len(repair.gold_context)):
                        if repair.gold_context[i] == "+|+":
                            onset = repair.gold_context[i+1]
                            break
                else:
                    gold_onset = ""
                    onset = ""
                    word = ""
                    if len(repair.gold_tags_right_context)>1:
                        gold_onset = repair.gold_tags_right_context[1]
                        onset = repair.tags_right_context[1]
                        word = repair.words_right_context[1]
                    #penult = repair.tags_left_context[-1]
                    #print repair
                    if k == "FP":
                        onset = gold_onset
                    if "<rps" in onset:
                        typedict["<rps"]+=1
                    elif "<e" in onset:
                        typedict["<e"]+=1
                    else:
                        if word in ["and","or","but","so","because","that","although"]:
                            typedict["CC"]+=1
                        elif word in ["i","we","they","im","ive","he","she","id"]:
                            typedict["subj"]+=1
                        elif word in ["you","the"] or "$" in word:
                            typedict["proper_other"]+=1
                        elif word in ["yeah","no","okay","yes","right","uh-huh"]:
                            typedict["ack"]+=1
                        elif word in ["it","its"]:
                            typedict["it"]+=1
                        else:
                            typedict[word]+=1
                         
                
                #if "<t" in repair.gold_context
                #for t in ["tt","cc","ct","tc"]:
                #    if "<" + t + ">" in onset:
                #        typedict[t]+=1
                #        if not t[0]=='t':
                #            print repair
                if tag == "<rps" and not k == 'FP':
                    lendict[repair.type]+=1
                error[k]['len'] = deepcopy(lendict)
                error[k]['type'] = deepcopy(typedict)

#tp = deepcopy(lendict)
#q1. THE RECALL RATES FOR VARIOUS GOLD REPAIRS
tp = error['TP']['len']
print tp
print error['FN']['len']
for k,v in sorted(error['FN']['len'].items()):
    print " & ".join([k, "({0})".format(v + tp[k]), 
                      '{0:.1f}'.format(100 * float(tp[k])/float(v+ tp[k]))]) + "\\\\"

tps = error['TP']['type']
fns = error['FN']['type']
fps = error['FP']['type']

total = sum(fns.values()+tps.values())

errormass = 0
errortotal = 0
for k,v in sorted(fns.items(),key= lambda x: x[1],reverse=True):
    print k,"&",v,"&",'{0:.2f}'.format(v/total * 100)
    errormass +=(v/total * 100)
    errortotal+=v
print "total &",errortotal,"&",'{0:.2f}'.format(errormass)

test_asr_RNN <type 'bool'>
test_RNN <type 'dict'>
heldout_asr_RNN <type 'bool'>
heldout_RNN <type 'dict'>
heldout_RNN t>

FP 834
left context=
right context=well|<e/><tt> i|<f/><tc> have|<f/><cc> to|<f/><cc> say|<f/><cc>
gold left context=
gold right context=well|<e/><tc> i|<f/><cc> have|<f/><cc> to|<f/><cc> say|<f/><cc>
type = None

TP 3264
left context=
right context=turn|<f/><ct> well|<e/><tt> i|<f/><tc> have|<f/><cc> to|<f/><cc>
gold left context=
gold right context=turn|<f/><ct> well|<e/><tc> i|<f/><cc> have|<f/><cc> to|<f/><cc>
type = None

FN 2468
left context=i|<f/><cc> really|<f/><cc> dont|<f/><cc> have|<f/><cc> a|<f/><cc>
right context=budget|<f/><cc> both|<f/><cc> my|<f/><cc> wife|<f/><cc> and|<f/><cc>
gold left context=i|<f/><cc> really|<f/><cc> dont|<f/><cc> have|<f/><cc> a|<f/><cc>
gold right context=budget|<f/><ct> both|<f/><tc> my|<f/><cc> wife|<f/><cc> and|<f/><cc>
type = None
heldout_RNN <rps
heldout_RNN <e
defaultdict(<type 'int'>, {})
defaultdict(<type 'int'>, {})
C

In [ ]:
 #TODO for paper/future
# - check WER for ASR results and exclude those with high ones given they might have high overlap :(
# - need to adjust the time to detection scores based on the time it comes in from Increco?? 
#      Also for ttdetection can only use word ends unless we re-do the mapping- just needs explanation
# - delayed accuracy based on time, or not bother? do moving window instead and plot this over time- average moving window accuracy
# - error analysis plots
# 036- full task with LSTM- should improve massively over 034, which also needs re-running
# Reproduce 027 (with full training data, efficiently) and re-run with LSTM- not much time.
#Q2 TODO the extent to which the network is memorizing- need to plug these in with the repair gold standards